In [7]:
from google.cloud import bigquery
import pandas

In [8]:
client = bigquery.Client(location="US")
print("Client creating using default project: {}".format(client.project))

Client creating using default project: ut-goog


### Query the columns from CFPB Complaints Dataset and return results in a Pandas dataframe

In [12]:
query = """
    SELECT 
      complaint_id,
      consumer_complaint_narrative,
      company_public_response,
      company_name,
      tags,
      consumer_consent_provided,
      submitted_via,
      date_sent_to_company,
      company_response_to_consumer,
      timely_response,
      consumer_disputed
    FROM `bigquery-public-data.cfpb_complaints.complaint_database`
    LIMIT 10
"""
query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
)  # API request - starts the query

complaints_df = query_job.to_dataframe()
complaints_df

### Calculate the percentage of null values in each column

In [14]:
query = """
    SELECT
      100*(COUNT(*) - COUNT(complaint_id)) / COUNT(*) AS complaint_id,
      100*(COUNT(*) - COUNT(consumer_complaint_narrative)) / COUNT(*) AS consumer_complaint_narrative,
      100*(COUNT(*) - COUNT(company_public_response)) / COUNT(*) AS company_public_response,
      100*(COUNT(*) - COUNT(company_name)) / COUNT(*) AS company_name,
      100*(COUNT(*) - COUNT(tags)) / COUNT(*) AS tags,
      100*(COUNT(*) - COUNT(consumer_consent_provided)) / COUNT(*) AS consumer_consent_provided,
      100*(COUNT(*) - COUNT(submitted_via)) / COUNT(*) AS submitted_via,
      100*(COUNT(*) - COUNT(date_sent_to_company)) / COUNT(*) AS date_sent_to_company,
      100*(COUNT(*) - COUNT(company_response_to_consumer)) / COUNT(*) AS company_response_to_consumer,
      100*(COUNT(*) - COUNT(timely_response)) / COUNT(*) AS timely_response,
      100*(COUNT(*) - COUNT(consumer_disputed)) / COUNT(*) AS consumer_disputed
    FROM `bigquery-public-data.cfpb_complaints.complaint_database`
"""
query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
)  # API request - starts the query

complaints_nulls = query_job.to_dataframe()
complaints_nulls

,complaint_id,consumer_complaint_narrative,company_public_response,company_name,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed
0,0.0,67.15356,62.422046,0.0,86.398418,1.182596,0.0,0.0,0.000067,0.0,48.874884


The columns - consumer_complaint_narrative, company_public_response and tags have a high percentage of NULL values.
Let's check if the consumer_complaint_narrative has enough number of rows to work with.

In [15]:
query = """
    SELECT
      SUM(CASE WHEN consumer_complaint_narrative IS NOT NULL THEN 1 ELSE 0 END) consumer_complaint_narrative,
      SUM(CASE WHEN company_public_response IS NOT NULL THEN 1 ELSE 0 END) company_public_response
    FROM `bigquery-public-data.cfpb_complaints.complaint_database`      
"""
query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
)  # API request - starts the query

complaints_non_null = query_job.to_dataframe()
complaints_non_null

,consumer_complaint_narrative,company_public_response
0,493726,564847


### Visualize the rows where the consumer_complaint_narrative is not null

In [17]:
query = """
    SELECT 
      *
    FROM `bigquery-public-data.cfpb_complaints.complaint_database`
    WHERE consumer_complaint_narrative IS NOT NULL
    LIMIT 100
"""

query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
)  # API request - starts the query

complaints_df = query_job.to_dataframe()
complaints_df

,date_received,product,subproduct,issue,subissue,consumer_complaint_narrative,company_public_response,company_name,state,zip_code,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed,complaint_id
0,2017-04-02,Prepaid card,General purpose card,Fees,None,I bought a {$400.00} prepaid Mastercard at XXX...,None,Blackhawk Network Holdings Inc.,OH,450XX,None,Consent provided,Web,2017-04-03,Closed with explanation,True,False,2415190
1,2017-01-29,Prepaid card,Electronic Benefit Transfer / EBT card,Fees,None,I receive monthly stipend for being a XXXX via...,None,Comerica,NJ,None,None,Consent provided,Web,2017-02-02,Closed with explanation,True,False,2316550
2,2015-12-24,Prepaid card,General purpose card,Fees,None,Without receiving any notification that my acc...,Company chooses not to provide a public response,U.S. BANCORP,NC,287XX,None,Consent provided,Web,2015-12-29,Closed with explanation,True,False,1716861
3,2015-10-13,Prepaid card,Government benefit payment card,Fees,None,Rush card was under maintance Monday XXXX XXXX...,None,"Empowerment Ventures, LLC",MI,490XX,None,Consent provided,Web,2015-11-02,Closed with non-monetary relief,True,False,1604786
4,2015-12-18,Prepaid card,Mobile wallet,Fees,None,Paypal reported 30 day delinquency against for...,Company has responded to the consumer and the ...,SYNCHRONY FINANCIAL,PA,19134,None,Consent provided,Web,2016-05-05,Closed with explanation,True,False,1708970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2016-08-15,Credit card,None,Other,None,My credit monitoring service has flagged a har...,Company has responded to the consumer and the ...,"CITIBANK, N.A.",MD,None,None,Consent provided,Web,2016-08-15,Closed with non-monetary relief,True,False,2063356
96,2016-07-25,Credit card,None,Other,None,Capitol One Bank XXXX The creditor did obtain ...,None,CAPITAL ONE FINANCIAL CORPORATION,CT,None,None,Consent provided,Web,2016-07-25,Closed with explanation,True,False,2029527
97,2016-10-31,Credit card,None,Other,None,CFPB complaint XX/XX/2016 This is request for ...,None,CAPITAL ONE FINANCIAL CORPORATION,AZ,856XX,None,Consent provided,Web,2016-10-31,Closed with explanation,True,True,2186763
98,2015-11-10,Credit card,None,Other,None,"Robocall from "" XXXX '' asking for information...",None,JPMORGAN CHASE & CO.,NJ,None,None,Consent provided,Web,2015-11-10,Closed with explanation,True,False,1648745
